The MIT License (MIT)

Copyright (c) 2021 NVIDIA CORPORATION

Permission is hereby granted, free of charge, to any person obtaining a copy of
this software and associated documentation files (the "Software"), to deal in
the Software without restriction, including without limitation the rights to
use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of
the Software, and to permit persons to whom the Software is furnished to do so,
subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS
FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR
COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER
IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [1]:
import pandas as pd
import pandas as cudf
from sklearn.model_selection import GroupKFold

pd.__version__, cudf.__version__

('1.1.5', '1.1.5')

In [2]:
from numba import cuda

def get_order_in_group(utrip_id_,order):
    for i in range(cuda.threadIdx.x, len(utrip_id_), cuda.blockDim.x):
        order[i] = i

def add_cumcount(df, sort_col, outputname):
    df = df.sort_values(sort_col, ascending=True)
    tmp = df[['utrip_id_', 'checkin']].groupby(['utrip_id_']).apply_grouped(
        get_order_in_group,incols=['utrip_id_'],
        outcols={'order': 'int32'},
        tpb=32)
    tmp.columns = ['utrip_id_', 'checkin', outputname]
    df = df.merge(tmp, how='left', on=['utrip_id_', 'checkin'])
    df = df.sort_values(sort_col, ascending=True)
    return(df)

In [3]:
train = cudf.read_csv('../../../data/train_set.csv').sort_values(by=['user_id','checkin'])
test = cudf.read_csv('../../../data/test_set.csv').sort_values(by=['user_id','checkin'])
#del train['Unnamed: 0']
#del test['row_num'], test['total_rows']

print(train.shape, test.shape)
train.head()

(1166835, 9) (378667, 9)


,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id
413669,29,2016-07-09,2016-07-11,47054,desktop,1601,Elbonia,Elbonia,29_1
413670,29,2016-07-11,2016-07-13,34444,desktop,1601,Elbonia,Elbonia,29_1
413671,29,2016-07-13,2016-07-16,12291,desktop,1601,Elbonia,Elbonia,29_1
413672,29,2016-07-16,2016-07-18,16386,desktop,8132,Elbonia,Elbonia,29_1
1128910,81,2016-05-15,2016-05-16,33665,desktop,9924,Elbonia,Elbonia,81_1


In [4]:
train['istest'] = 0
test['istest'] = 1
raw = cudf.concat([train,test], sort=False )
raw = raw.sort_values( ['user_id','checkin'], ascending=True )
raw.head()

,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id,istest
413669,29,2016-07-09,2016-07-11,47054,desktop,1601,Elbonia,Elbonia,29_1,0
413670,29,2016-07-11,2016-07-13,34444,desktop,1601,Elbonia,Elbonia,29_1,0
413671,29,2016-07-13,2016-07-16,12291,desktop,1601,Elbonia,Elbonia,29_1,0
413672,29,2016-07-16,2016-07-18,16386,desktop,8132,Elbonia,Elbonia,29_1,0
355509,65,2016-09-26,2016-09-29,36403,desktop,3577,The Devilfire Empire,Cobra Island,65_1,1


In [5]:
raw['fold'] = 0
group_kfold = GroupKFold(n_splits=5)
for fold, (train_index, test_index) in enumerate(group_kfold.split(X=raw, y=raw, groups=raw['utrip_id'])):
    raw.iloc[test_index,10] = fold

raw['fold'].value_counts()

1    309101
0    309101
4    309100
3    309100
2    309100
Name: fold, dtype: int64

In [6]:
#This flag tell which row must be part of the submission file.

raw['submission'] = 0
raw.loc[ (raw.city_id==0)&(raw.istest) ,'submission'] = 1

raw.loc[ raw.submission==1 ]

,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id,istest,fold,submission
355513,65,2016-10-03,2016-10-04,0,mobile,4132,The Devilfire Empire,NaN,65_1,1,2,1
356899,67,2016-08-11,2016-08-14,0,desktop,9924,Tcherkistan,NaN,67_1,1,1,1
10963,115,2016-04-06,2016-04-07,0,desktop,9924,Elbonia,NaN,115_1,1,0,1
120565,279,2016-03-27,2016-04-01,0,desktop,2803,Tcherkistan,NaN,279_1,1,3,1
139366,307,2016-06-02,2016-06-03,0,desktop,8132,Elbonia,NaN,307_1,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
353170,6257974,2016-07-05,2016-07-07,0,tablet,7974,The Devilfire Empire,NaN,6257974_1,1,2,1
353174,6258010,2016-06-10,2016-06-12,0,mobile,359,The Devilfire Empire,NaN,6258010_1,1,0,1
353181,6258104,2016-08-25,2016-08-27,0,mobile,359,Gondal,NaN,6258104_4,1,1,1
353186,6258120,2016-07-24,2016-07-25,0,desktop,9924,Gondal,NaN,6258120_1,1,4,1


In [7]:
#number of places visited in each trip

aggs = raw.groupby('utrip_id', as_index=False)['user_id'].count()
aggs.columns = ['utrip_id', 'N']
raw = raw.merge(aggs, on=['utrip_id'], how='inner')

In [8]:
raw['utrip_id_'], mp = raw['utrip_id'].factorize()

In [14]:
raw['checkin'] = pd.to_datetime(raw.checkin, format="%Y-%m-%d")
raw['checkout'] = pd.to_datetime(raw.checkout, format="%Y-%m-%d")

In [18]:
raw["dcount"] = raw.groupby(["utrip_id_"])["checkin"].rank(ascending=True).astype(int) - 1
raw["icount"] = raw["N"] - raw["dcount"] - 1

In [19]:
# raw = add_cumcount(raw, ['utrip_id_','checkin'], 'dcount')

In [20]:
raw.head(50)

,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id,istest,fold,submission,N,utrip_id_,dcount,icount
0,29,2016-07-09,2016-07-11,47054,desktop,1601,Elbonia,Elbonia,29_1,0,3,0,4,0,0,3
1,29,2016-07-11,2016-07-13,34444,desktop,1601,Elbonia,Elbonia,29_1,0,3,0,4,0,1,2
2,29,2016-07-13,2016-07-16,12291,desktop,1601,Elbonia,Elbonia,29_1,0,3,0,4,0,2,1
3,29,2016-07-16,2016-07-18,16386,desktop,8132,Elbonia,Elbonia,29_1,0,3,0,4,0,3,0
4,65,2016-09-26,2016-09-29,36403,desktop,3577,The Devilfire Empire,Cobra Island,65_1,1,2,0,5,1,0,4
5,65,2016-09-29,2016-10-01,23921,desktop,8223,The Devilfire Empire,Cobra Island,65_1,1,2,0,5,1,1,3
6,65,2016-10-01,2016-10-02,42682,mobile,4132,The Devilfire Empire,Cobra Island,65_1,1,2,0,5,1,2,2
7,65,2016-10-02,2016-10-03,29990,mobile,4132,The Devilfire Empire,Cobra Island,65_1,1,2,0,5,1,3,1
8,65,2016-10-03,2016-10-04,0,mobile,4132,The Devilfire Empire,NaN,65_1,1,2,1,5,1,4,0
9,67,2016-07-30,2016-08-01,64876,desktop,9924,Tcherkistan,Fook Island,67_1,1,1,0,8,2,0,7


In [22]:
raw.to_parquet( '../00_Data/train_and_test_2.pq')